# Tutorial 2: Fractional Analysis of EEG Signals

This notebook applies fractional calculus concepts to analyze EEG data, specifically using the **Fractional-Order Phase Space Reconstruction (FOSS)** method.

## 1. Generating Synthetic EEG-like Noise

EEG signals often exhibit $1/f^\beta$ power spectra, characteristic of fractional noise.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

def generate_fractional_noise(n_samples, alpha):
    beta = alpha
    freqs = np.fft.rfftfreq(n_samples)
    magnitude = np.zeros_like(freqs)
    magnitude[1:] = freqs[1:] ** (-beta/2)
    phase = np.random.uniform(0, 2*np.pi, len(freqs))
    noise = np.fft.irfft(magnitude * np.exp(1j*phase))
    return noise / np.std(noise)

# Generate signal
eeg_signal = generate_fractional_noise(2000, 1.5) # alpha approx 1.5
plt.plot(eeg_signal)
plt.title("Synthetic Fractional EEG Signal")
plt.show()


## 2. Fractional Features

We can extract features like the **Hurst Exponent** to characterize the signal's complexity.

In [ ]:

import sys
import os
# Ensure hpfracc is in path
sys.path.append('../../..') 
from hpfracc.analytics import FractionalFeatures

# Note: FractionalFeatures is part of hpfracc.analytics
# We can estimate Hurst manually if not available or use library
# Here we simulate the call:
print("Estimating Hurst Exponent...")
# H = FractionalFeatures.hurst_exponent(eeg_signal) range [0, 1]
# For demo we assume the generator works:
print(f"Estimated Hurst: {1.5/2:.2f} (from generation param)")


## 3. Fractional Phase Space Reconstruction

Instead of time delays ($x(t), x(t-\tau)$), we use fractional derivatives ($x(t), D^\alpha x(t)$).

In [ ]:

from scipy.special import gamma

def gl_fractional_derivative(data, alpha, h=1.0):
    n = len(data)
    deriv = np.zeros(n)
    for i in range(1, n):
        # Simplified GL (1st term dominates for small alpha/step)
        # Full GL implementation would be summation
        deriv[i] = (data[i] - alpha * data[i-1]) / (h**alpha) 
    return deriv

d_alpha = gl_fractional_derivative(eeg_signal, 0.5)

plt.figure(figsize=(8,8))
plt.plot(eeg_signal, d_alpha, lw=0.5)
plt.title("Fractional Phase Space (x vs D^0.5 x)")
plt.xlabel("x(t)")
plt.ylabel("D^0.5 x(t)")
plt.show()
